In [1]:
from torchvision import models, transforms
from torch.optim import SGD
from tqdm import tqdm
from torch.utils.data import DataLoader
import torch.nn
from dotenv import load_dotenv
import os
from datahandling.transforms import to_numeric_label, to_class_int

from dataset.chestxray import ChestXRayDataset

/cluster/home/larsira/.local/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


# Settings

In [2]:
device = "cuda"

transform = transforms.Compose([
    transforms.RandomHorizontalFlip(p=0.5),
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
])

optimizer = SGD
cel = torch.nn.CrossEntropyLoss(reduction="mean")

# Load Environment variables

In [4]:
load_dotenv()
db_path = os.getenv("DB_PATH")
img_dir_name = os.getenv("IMG_DIR")

annotation_folder = os.path.join(db_path, "training_sets/chexnet")
target_labels = ChestXRayDataset.target_labels
img_dir = os.path.join(db_path, img_dir_name)

# Load Data

In [5]:
batch_size = 8

train_set = ChestXRayDataset(os.path.join(annotation_folder, "train.csv"), img_dir, read_lib="pil", transform=transform, target_transform=to_numeric_label)
val_set = ChestXRayDataset(os.path.join(annotation_folder, "validation.csv"), img_dir, read_lib="pil", transform=transform, target_transform=to_numeric_label)
test_set = ChestXRayDataset(os.path.join(annotation_folder, "test.csv"), img_dir, read_lib="pil", transform=transform, target_transform=to_numeric_label)

train_loader = DataLoader(train_set, batch_size)
val_loader = DataLoader(val_set, batch_size)
test_set = DataLoader(test_set, batch_size)

train_sample = next(iter(train_loader))
print(train_sample[0][0].unsqueeze(0).shape)

torch.Size([1, 3, 224, 224])


In [6]:
num_epochs = 1000
model = models.densenet121(progress=True).to(device)

pred = model(train_sample[0].to(device))

RuntimeError: Unable to find a valid cuDNN algorithm to run convolution

In [55]:
for epoch in range(num_epochs):
    print("Epoch {} of {}".format(epoch+1, num_epochs))

    model.train()
    

Epoch 1 of 1000
Epoch 2 of 1000
Epoch 3 of 1000
Epoch 4 of 1000
Epoch 5 of 1000
Epoch 6 of 1000
Epoch 7 of 1000
Epoch 8 of 1000
Epoch 9 of 1000
Epoch 10 of 1000
Epoch 11 of 1000
Epoch 12 of 1000
Epoch 13 of 1000
Epoch 14 of 1000
Epoch 15 of 1000
Epoch 16 of 1000
Epoch 17 of 1000
Epoch 18 of 1000
Epoch 19 of 1000
Epoch 20 of 1000
Epoch 21 of 1000
Epoch 22 of 1000
Epoch 23 of 1000
Epoch 24 of 1000
Epoch 25 of 1000
Epoch 26 of 1000
Epoch 27 of 1000
Epoch 28 of 1000
Epoch 29 of 1000
Epoch 30 of 1000
Epoch 31 of 1000
Epoch 32 of 1000
Epoch 33 of 1000
Epoch 34 of 1000
Epoch 35 of 1000
Epoch 36 of 1000
Epoch 37 of 1000
Epoch 38 of 1000
Epoch 39 of 1000
Epoch 40 of 1000
Epoch 41 of 1000
Epoch 42 of 1000
Epoch 43 of 1000
Epoch 44 of 1000
Epoch 45 of 1000
Epoch 46 of 1000
Epoch 47 of 1000
Epoch 48 of 1000
Epoch 49 of 1000
Epoch 50 of 1000
Epoch 51 of 1000
Epoch 52 of 1000
Epoch 53 of 1000
Epoch 54 of 1000
Epoch 55 of 1000
Epoch 56 of 1000
Epoch 57 of 1000
Epoch 58 of 1000
Epoch 59 of 1000
Epoch 